<a href="https://colab.research.google.com/github/valuntiny/kaggle_intermediate_ML/blob/master/kaggle_intermediate_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**Guojing Wu** *| 2019-07-31*

<a href = "https://www.kaggle.com/alexisbcook/pipelines"> Kaggle: Intermediate Machine Learning </a>

# Objectives

* missing value

* pipelines

* CV

* XGBoost

* leakage

# pipeline